In [1]:
# Library to suppress warnings or deprecation notes 
import warnings
warnings.filterwarnings('ignore')

import os
import math

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm

# Import the required library
from geopy.geocoders import Nominatim 
from unidecode import unidecode
import difflib 
import numpy as np

In [2]:
def process_decode(x, key):
    return 'UNKNOWN' if x[key] != x[key] else unidecode(x[key]).upper()

In [3]:
# Air pollution file
air_filename = '../../../src/data/data_processed/merged_air_quality_data_CSV/thayes_air_quality_stations_with_powiat_missing3.csv'
air_df = pd.read_csv(air_filename)

In [4]:
air_df.head()

,No,Station code,International code,Station name,Old station code,Launching date,Closing date,Station type,Region type,Station model,Voivodeship,City,Address,WGS84 φ N,WGS84 λ E,county,STANAME
0,1,DsBialka,NaN,Białka,DsBialka,1990-01-03,2005-12-31,industrial,suburban,container,DOLNOŚLĄSKIE,Białka,NaN,51.197783,16.117390,powiat legnicki,BIALKA
1,2,DsBielGrot,NaN,Bielawa - ul. Grota Roweckiego,DsBielGrot,1994-01-02,2003-12-31,background,urban,in-house,DOLNOŚLĄSKIE,Bielawa,ul. Grota Roweckiego 6,50.682510,16.617348,powiat dzierżoniowski,BIELAWA
2,3,DsBogatFrancMOB,PL0602A,Bogatynia Mobil,DsBogatMob,2015-01-01,2015-12-31,background,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,ul. Francuska/Kręta,50.940998,14.916790,powiat zgorzelecki,BOGATYNIA
3,4,DsBogChop,PL0315A,Bogatynia - Chopina,DsBogChop,1996-01-01,2013-12-31,industrial,urban,container,DOLNOŚLĄSKIE,Bogatynia,ul. Chopina 35,50.905856,14.967175,powiat zgorzelecki,BOGATYNIA
4,5,DsBogZatonieMob,PL0576A,Bogatynia - Mobil,DsBogZatonieMob,2012-01-01,2012-12-31,industrial,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,"ul. Konrada, Zatonie",50.943245,14.913327,powiat zgorzelecki,BOGATYNIA


In [5]:
air_df[air_df['county'].isnull()]

,No,Station code,International code,Station name,Old station code,Launching date,Closing date,Station type,Region type,Station model,Voivodeship,City,Address,WGS84 φ N,WGS84 λ E,county,STANAME
506,507,MpSzarowWIOS1901,NaN,Szarów1901,MpSzarowWIOS1901,2002-01-01,2002-12-31,background,suburban,NO DATA,MAŁOPOLSKIE,Szarów,ul. Szarów 241,-999.0000,-999.0000,NaN,SZAROW
566,567,MzPlockPKN3,NaN,Płock-Chełpowo,MzPlockPKN3,1993-01-01,2005-12-31,industrial,urban,container,MAZOWIECKIE,Chełpowo,ul. Długa,52.5675,19.6875,NaN,CHELPOWO
929,930,SlRacibRaci_boruc,NaN,Racibórz_borucin-WSSE,SlRacibRaci_boruc,2005-01-01,2007-12-31,background,urban,container,ŚLĄSKIE,Borucin,St.Uzd.Wody,-999.0000,-999.0000,NaN,BORUCIN


In [6]:
# Drop rows with null county
air_df.dropna(subset=['county'], inplace=True)

In [7]:
air_df['county'].isnull().sum()

0

In [8]:
# Get a list of clean counties for air_df
powiat_air = air_df['county'].unique().tolist()
powiat_air = list(map(lambda x: unidecode(x).upper(), powiat_air))
powiat_air

['POWIAT LEGNICKI',
 'POWIAT DZIERZONIOWSKI',
 'POWIAT ZGORZELECKI',
 'POWIAT BOLESLAWIECKI',
 'POWIAT GLOGOWSKI',
 'POWIAT KLODZKI',
 'POWIAT LUBANSKI',
 'POWIAT TRZEBNICKI',
 'POWIAT JAWORSKI',
 'POWIAT WALBRZYSKI',
 'JELENIA GORA COUNTY',
 'POWIAT POLKOWICKI',
 'POWIAT KAMIENNOGORSKI',
 'POWIAT WROCLAWSKI',
 'POWIAT SWIDNICKI',
 'POWIAT KARKONOSKI',
 'LEGNICA',
 'POWIAT LUBINSKI',
 'POWIAT LWOWECKI',
 'POWIAT MILICKI',
 'POWIAT OLAWSKI',
 'POWIAT OLESNICKI',
 'POWIAT SREDZKI',
 'POWIAT STRZELINSKI',
 'WALBRZYCH COUNTY',
 'POWIAT WROCLAWSKI',
 'POWIAT ZABKOWICKI',
 'POWIAT ZLOTORYJSKI',
 'POWIAT INOWROCLAWSKI',
 'POWIAT BRODNICKI',
 'BYDGOSZCZ',
 'POWIAT BYDGOSKI',
 'POWIAT CHELMINSKI',
 'POWIAT ALEKSANDROWSKI',
 'POWIAT WLOCLAWSKI',
 'POWIAT TORUNSKI',
 'POWIAT LIPNOWSKI',
 'POWIAT GOLUBSKO-DOBRZYNSKI',
 'POWIAT GRUDZIADZKI',
 'POWIAT GRUDZIADZKI',
 'POWIAT RADZIEJOWSKI',
 'POWIAT RYPINSKI',
 'POWIAT MOGILENSKI',
 'POWIAT NAKIELSKI',
 'POWIAT ZNINSKI',
 'POWIAT SEPOLENSKI',
 'POWIAT

In [9]:
pl_filename = '../../../src/data/data_auxiliary/PL.txt'
pl_df = pd.read_csv(pl_filename, sep="\t")

In [10]:
pl_df.head()

,country code,postal code,place name1,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,PL,59-700,Rakowice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.2447,15.5347,6
1,PL,59-700,Chościszowice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.2868,15.5891,6
2,PL,59-700,Golnice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3504,15.5852,6
3,PL,59-700,Dąbrowa Bolesławiecka,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3264,15.5772,6
4,PL,59-700,Parkoszów,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3753,15.5811,6


In [11]:
pl_df[pl_df['admin name2'].isnull()]

,country code,postal code,place name1,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
8809,PL,88-420,Jeziora,Kujawsko-Pomorskie,73,NaN,NaN,NaN,NaN,52.6545,17.7599,6


In [12]:
# Drop rows with null admin name2
pl_df.dropna(subset=['admin name2'], inplace=True)

In [13]:
powiat_pl = pl_df['admin name2'].unique().tolist()
powiat_pl = list(map(lambda x: unidecode(x).upper(), powiat_pl))

In [14]:
intersect = list(set(powiat_air) - set(powiat_pl))
intersect

['POWIAT LESZCZYNSKI',
 'POWIAT KLODZKI',
 'POWIAT OLAWSKI',
 'POWIAT OLESNICKI',
 'POWIAT GLUBCZYCKI',
 'POWIAT TRZEBNICKI',
 'POWIAT LWOWECKI',
 'POWIAT LUBANSKI',
 'POWIAT MILICKI',
 'POWIAT KAMIENNOGORSKI',
 'POWIAT ZDUNSKOWOLSKI',
 'POWIAT CHELMSKI',
 'POWIAT TORUNSKI',
 'POWIAT GLOGOWSKI',
 'POWIAT ZGORZELECKI',
 'POWIAT WALBRZYSKI',
 'POWIAT KARKONOSKI',
 'POWIAT STASZOWSKI',
 'POWIAT KONINSKI',
 'POWIAT ZABKOWICKI',
 'POWIAT GDANSKI',
 'POWIAT JAWORSKI',
 'POWIAT POLKOWICKI',
 'POWIAT WARSZAWSKI ZACHODNI',
 'POWIAT WODZISLAWSKI',
 'POWIAT STRZELINSKI',
 'POWIAT RADOMSKI',
 'POWIAT ZLOTORYJSKI',
 'POWIAT LUBINSKI']

In [15]:
air_powiat_mapping = {'POWIAT POLKOWICKI': 'POLKOWICE COUNTY',
 'POWIAT GLOGOWSKI': 'GLOGOW COUNTY',
 'POWIAT ZGORZELECKI': 'ZGORZELEC COUNTY',
 'POWIAT JAWORSKI': 'JAWOR COUNTY',
 'POWIAT KLODZKI': 'KLODZKO COUNTY',
 'POWIAT GDANSKI': 'GDANSK COUNTY',
 'POWIAT STRZELINSKI': 'STRZELIN COUNTY',
 'POWIAT WARSZAWSKI ZACHODNI': 'WARSAW WEST COUNTY',
 'POWIAT LWOWECKI': 'LWOWEK SLASKI COUNTY',
 'POWIAT MILICKI': 'MILICZ COUNTY',
 'POWIAT OLESNICKI': 'OLESNICA COUNTY',
 'POWIAT WODZISLAWSKI': 'WODZISLAW COUNTY',
 'POWIAT TORUNSKI': 'TORUN COUNTY',
 'POWIAT RADOMSKI': 'RADOM COUNTY',
 'POWIAT CHELMSKI': 'CHELM COUNTY',
 'POWIAT LESZCZYNSKI': 'LESZNO_COUNTY',
 'POWIAT STASZOWSKI': 'STASZOW COUNTY',
 'POWIAT OLAWSKI': 'OLAWA COUNTY',
 'POWIAT TRZEBNICKI': 'TRZEBNICA COUNTY',
 'POWIAT KAMIENNOGORSKI': 'KAMIENNA GORA COUNTY',
 'POWIAT GLUBCZYCKI': 'GLUBCZYCE COUNTY',
 'POWIAT KONINSKI': 'KONIN COUNTY',
 'POWIAT LUBANSKI': 'LUBAN COUNTY',
 'POWIAT LUBINSKI': 'LUBIN COUNTY',
 'POWIAT ZABKOWICKI': 'ZABKOWICE SLASKIE COUNTY',
 'POWIAT WALBRZYSKI': 'WALBRZYCH COUNTY',
 'POWIAT KARKONOSKI': 'JELENIA GORA COUNTY',
 'POWIAT ZLOTORYJSKI': 'ZLOTORYJA COUNTY',
 'POWIAT ZDUNSKOWOLSKI': 'ZDUNSKA WOLA COUNTY'}

for i, powiat in enumerate(powiat_air):
    if (powiat in air_powiat_mapping):
        powiat_air[i] = air_powiat_mapping[powiat]

In [16]:
intersect = list(set(powiat_air) - set(powiat_pl))
intersect

[]

In [17]:
# Read in all the powiats from static data and compare to PL.txt
root_directory = '../../../src/data/data_raw/static_annual_data/2010-2021'
directory_mask = os.path.join(root_directory, '*powiat*.xlsx')
# Find list of matching filenames in the directory
filenames = glob.glob(directory_mask)

powiat_static_list = []
for file in filenames:
    print('Process {}'.format(file))
    df = pd.read_excel(file, sheet_name='TABLE')
    new_list = df.apply(lambda row: process_decode(row, 'Name'), axis = 1).unique().tolist()
    powiat_static_list = powiat_static_list + new_list
    
powait_static_list = set(powiat_static_list)

Process ../../../src/data/data_raw/static_annual_data/2010-2021/area_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/emission_of_particulates_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/emission_of_pollutant_gases_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/forest_area_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/population_density_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/share_of_green_areas_by_powiat_2010-2021.xlsx
Process ../../../src/data/data_raw/static_annual_data/2010-2021/total_vehicles_by_type_by_powiat_2010-2021.xlsx


In [18]:
intersect = list(set(powiat_static_list) - set(powiat_pl))
intersect

['CITY WITH POWIAT STATUS TARNOBRZEG',
 'CITY WITH POWIAT STATUS RUDA SLASKA',
 'CITY WITH POWIAT STATUS JELENIA GORA',
 'CITY WITH POWIAT STATUS LOMZA',
 'CITY WITH POWIAT STATUS WALBRZYCH TO 2002',
 'UNKNOWN',
 'POWIAT WOLOWSKI',
 'CITY WITH POWIAT STATUS BIALYSTOK',
 'POWIAT WARSZAWSKI',
 'POWIAT TRZEBNICKI',
 'CITY WITH POWIAT STATUS SWIETOCHLOWICE',
 'POWIAT ZDUNSKOWOLSKI',
 'CITY WITH POWIAT STATUS BIELSKO-BIALA',
 'POWIAT GLOGOWSKI',
 'CITY WITH POWIAT STATUS RADOM',
 'POWIAT WALBRZYSKI',
 'CITY WITH POWIAT STATUS PIOTRKOW TRYBUNALSKI',
 'CITY WITH POWIAT STATUS KONIN',
 'POWIAT LOMZYNSKI',
 'CITY WITH POWIAT STATUS LEGNICA',
 'CITY WITH POWIAT STATUS GDANSK',
 'CITY WITH POWIAT STATUS SOPOT',
 'CITY WITH POWIAT STATUS BYTOM',
 'CITY WITH POWIAT STATUS LODZ',
 'POWIAT WARSZAWSKI ZACHODNI',
 'CITY WITH POWIAT STATUS ZIELONA GORA',
 'POWIAT GOROWSKI',
 'CITY WITH POWIAT STATUS SUWALKI',
 'POWIAT ZLOTORYJSKI',
 'CITY WITH POWIAT STATUS WROCLAW',
 'CITY WITH POWIAT STATUS CHELM',
 '

In [19]:
powiat_static_mapping = {'POWIAT RZESZOWSKI': 'RZESZOW COUNTY',
 'POWIAT LOMZYNSKI': 'LOMZA COUNTY',
 'POWIAT GOROWSKI': 'GORA COUNTY',
 'POWIAT WARSZAWSKI': 'WARSAW WEST COUNTY',
 'POWIAT WOLOWSKI': 'WOLOW COUNTY'}
# Merge the two dictionaries
air_powiat_mapping.update(powiat_static_mapping)
overall_mapping = air_powiat_mapping

# Clean up the extra data in some of the fields 
powiat_static_list = [i.replace('CITY WITH POWIAT STATUS ', '').replace(' SINCE 2013', '')
                      .replace(' TO 2002', '').replace('CAPITAL CITY ', '') for i in powiat_static_list]

for i, powiat in enumerate(powiat_static_list):
    if (powiat in overall_mapping):
        powiat_static_list[i] = overall_mapping[powiat]


In [20]:
intersect = list(set(powiat_static_list) - set(powiat_pl))
intersect

['UNKNOWN']

In [21]:
stations_filename = '../../../src/data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod.csv'
stations_df = pd.read_csv(stations_filename)

In [22]:
stations_df.head()

,STAID,STANAME,CN,LAT,LON,HGHT,county,state,Voivodeship
0,24860,PSZCZYNA,PL,50.000000,18.916667,261,powiat pszczyński,województwo śląskie,Silesian
1,24876,JABLONKA,PL,49.466667,19.700000,671,powiat nowotarski,województwo małopolskie,Lesser Poland
2,24880,GUBALOWKA,PL,49.300000,19.933333,856,powiat tatrzański,województwo małopolskie,Lesser Poland
3,24881,WITOW,PL,49.333333,19.833333,844,powiat tatrzański,województwo małopolskie,Lesser Poland
4,24885,LAZY,PL,49.966667,20.500000,251,powiat bocheński,województwo małopolskie,Lesser Poland


In [23]:
stations_df['county'].isnull().sum()

0

In [24]:
powiat_stations = stations_df['county'].unique().tolist()
powiat_stations = list(map(lambda x: unidecode(x).upper(), powiat_stations))

In [25]:
intersect = list(set(powiat_stations) - set(powiat_pl))
intersect

['POWIAT STASZOWSKI',
 'POWIAT LESZCZYNSKI',
 'POWIAT KLODZKI',
 'KRALOVEHRADECKY KRAJ',
 'POWIAT RZESZOWSKI',
 'POWIAT KARKONOSKI']

In [26]:
#Update the mapping dictionry
overall_mapping.update({'KRALOVEHRADECKY KRAJ': 'JELENIA GORA COUNTY'})

for i, powiat in enumerate(powiat_stations):
    if (powiat in overall_mapping):
        powiat_stations[i] = overall_mapping[powiat]

In [27]:
intersect = list(set(powiat_stations) - set(powiat_pl))
intersect

[]